In [1]:
# Load AIEEE Data

In [2]:
!nvidia-smi

Sat Mar 19 01:47:31 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.86       Driver Version: 470.86       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   42C    P0    49W / 250W |   5988MiB / 16280MiB |     27%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"    
import tensorflow as tf

In [4]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


import sys  
sys.path.insert(0, 'Models/CBSEData/CBSEPreprocessing/')
from parse_cbse import *

In [6]:
cbse_df = CBSEData()

Load Models

In [7]:
#  other models
AIEEE_BERT = "Models/AIEEEGenderData/BERT/SavedModel/BERT"
CBSE_BERT = "Models/CBSEData/BERT/SavedModel/BERT"
ER_BERT = "Models/ERData/BERT/SavedModel/BERT"

In [8]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [9]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [10]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_BERT)

In [11]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_BERT)

In [12]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_BERT)

In [13]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


Predict Gender

In [14]:
from transformers import pipeline
from sklearn.metrics import classification_report


In [18]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
#     import pdb; pdb.set_trace()
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    print(classification_report(true, labels, digits = 4))
    return 

In [ ]:
print_results(cbse_model, tokenizer, cbse_df)

In [ ]:
print_results(aieee_model, tokenizer, cbse_df)

In [19]:
print_results(er_model, tokenizer, cbse_df)

              precision    recall  f1-score   support

         0.0     0.9813    0.9564    0.9687     67970
         1.0     0.9657    0.9853    0.9754     84519

    accuracy                         0.9725    152489
   macro avg     0.9735    0.9709    0.9720    152489
weighted avg     0.9726    0.9725    0.9724    152489

